# Análisis Estadístico

En la celda de abajo está el código, para revisar qué y cómo hace lo que hace. 

In [36]:
#!/usr/bin/env python3
# -*- coding: utf8 -*-

"""Analisis estadístico de la densidad de cursos en cada día de la semana"""
import sys
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import parser_horarios

# Esto hace que botonee si lee mal alguna linea del CSV
# Generalmente, tiene algun encabezado o algo
parser_horarios.debug = False

class Grilla:
    # Esta clase contiene una matriz con la cantidad de cursos que se dan en cada horario
    # La idea, es que con la funcion cargar_curso() le puedas ir agregando uno por uno
    # Faltaría una función descargar_curso(), pero sería trivial hacerla :)
    
    # La grilla por defecto empieza a las 7am y termina a las 23
    # Si hay materias de media hora, se puede usar el parametro fracciones_hora=2 para dividir cada 30 min
    # Si la grilla es posee separaciones de 1h y una materia termina 17:30, se la considera que termina a las 18
    # La primer fila de la matriz es el horario de [7, 8), la segunda de [8,9)hs y así.
    def __init__(self, hora_inicio=7, hora_fin=23, fracciones_hora=1):
        """Las horas son [inicio, fin)"""
        self.hora_inicio = hora_inicio
        self.hora_fin = hora_fin
        self.fracciones_hora = fracciones_hora
        self.cantidad_dias_semana = 6
        
        self.grilla = np.zeros( (int((self.hora_fin-self.hora_inicio)*self.fracciones_hora), self.cantidad_dias_semana), dtype=int )

    def __discretizar_hora(self, hora):
        if hora > self.hora_fin:
            raise ValueError("Hora debe ser menor que "+self.hora_fin)
        # Acá elijo si redondeo para arriba o abajo. Cambiar round por int para que ande distinto
        return round((hora-self.hora_inicio)*self.fracciones_hora)
    
    def cargar_curso(self, curso):
        for h in curso.horario:
            self.cargar_horario(h)
    
    def cargar_horario(self, horario):
        dia=horario[0]
        inicio=horario[1]
        fin=horario[2]
        # Resto uno en día porque arranca en 1 (Arreglar... algún día)
        # Notar que no estoy verificando NADA
        self.grilla[self.__discretizar_hora(inicio):self.__discretizar_hora(fin), dia-1] += 1

    def __str__(self):
        return "De ["+str(self.hora_inicio)+", " +str(self.hora_fin) + ")hs\nL M X J V S\n" + str(self.grilla)
        
    def graficar(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        # Hasta acá llegó mi amor
        raise NotImplementedError('No entiendo matplotlib')
        
    

def cargar_planilla(str_archivo='Horarios_1Q2016.csv'):
    # Cargo datos
    archivo_datos = 'Horarios_1Q2016.csv'
    f = open(archivo_datos, 'rb')
    str_data = f.read().decode('utf8', 'ignore') # Esto no me gusta
    f.close()
    
    return parser_horarios.procesar_data(str_data)

Ahora sólo voy a filtrar los datos que me importan con esta función.
Puede usarse otro criterio para, por ejemplo, mostrar sólo las electivas (pero habría que armar una lista)

In [37]:
def crear_grilla_ocupacion(cursos):
    # curso.codigo es un entero con el codigo del curso, ejemplo 6606 o 8610
    grilla = Grilla(fracciones_hora=1)
    i=0
    for curso in cursos:
        # Selecciono los del depto de electrónica plan '86
        if int(curso.codigo/100)==66:
            i=i+1
            grilla.cargar_curso(curso)
    
    print("Cantidad de cursos: %d"%i, file=sys.stderr)    
    
    return grilla

Y acá abajo cargo los datos e imprimo la grilla. (Ir a Files para ver los CSV disponibles)

In [38]:
cursos = cargar_planilla('Horarios_1Q2016.csv')
grilla = crear_grilla_ocupacion(cursos)
    
print(grilla)

De [7, 23)hs
L M X J V S
[[ 0  0  0  0  0  1]
 [ 0  1  0  1  0  1]
 [ 0  2  1  1  0  1]
 [ 0  2  2  3  1  2]
 [ 0  1  2  3  1  2]
 [ 0  0  1  3  1  1]
 [ 2  0  0  3  0  0]
 [ 2  0  0  4  0  0]
 [ 2  1  1  3  0  0]
 [21 13  8 12 12  0]
 [23 16  8 11 12  0]
 [23 16  7 11 12  0]
 [20 22 20 21 19  0]
 [20 22 20 21 19  0]
 [20 21 20 19 18  0]
 [ 3  3  3  3  2  0]]


Cantidad de cursos: 95


Ahora voy a cargar las materias optativas y analizar de nuevo

In [39]:
archivo_datos = 'optativas.csv'
f = open(archivo_datos, 'r')
lst_optativas = f.read().replace(".", "").split()
f.close()

def crear_grilla_ocupacion2(cursos):
    # curso.codigo es un entero con el codigo del curso, ejemplo 6606 o 8610
    grilla = Grilla(fracciones_hora=1)
    i=0
    for curso in cursos:
        # Selecciono los del depto de electrónica plan '86
        if lst_optativas.count(str(curso.codigo)) != 0:
            i=i+1
            grilla.cargar_curso(curso)
    
    print("Cantidad de cursos: %d"%i, file=sys.stderr)    
    
    return grilla

grilla2 = crear_grilla_ocupacion2(cursos)

print(grilla2)

De [7, 23)hs
L M X J V S
[[ 0  0  0  0  1  0]
 [ 1  1  0  0  2  0]
 [ 1  2  1  0  1  0]
 [ 1  2  2  2  2  2]
 [ 0  1  2  2  1  2]
 [ 0  0  1  3  1  1]
 [ 1  0  0  3  0  0]
 [ 1  1  2  7  0  0]
 [ 1  3  2  6  0  0]
 [17  8  8 14  8  0]
 [19  8  7 10  9  0]
 [21 10  8 13  9  0]
 [18 20 19 17 12  0]
 [18 20 19 17 11  0]
 [16 16 18 14 10  0]
 [ 6  3  3  1  3  0]]


Cantidad de cursos: 83
